<p style="text-align: center;"><img src="https://docs.google.com/uc?id=1lY0Uj5R04yMY3-ZppPWxqCr5pvBLYPnV" class="img-fluid" alt="Rossum"></p>

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from sklearn.model_selection import train_test_split
from keras import optimizers
from keras.preprocessing import image
import matplotlib.pyplot as plt
from matplotlib.image import imread

In [ ]:
pwd

# Dataset Info

The Dogs vs. Cats dataset is a common computer vision dataset in which pictures are classified as either including a dog or a cat.

After the dataset is well studied, it can be used to understand and practice how to design, evaluate, and apply convolutional neural networks for image classification.

You will build a classifier with images and try to detect dogs versus cats using CNN.

Train set includes 12500 cat-5026 dog images, validation set includes 1219 cat-1071 dog images and test set incgludes 6897 cat and dogs images together. 

In [ ]:
my_data_dir= "/kaggle/input/dog-cat/data/"

In [ ]:
os.listdir(my_data_dir)

In [ ]:
val_path = my_data_dir+'validation'
train_path = my_data_dir+'train'
test_path = my_data_dir+'test'

In [ ]:
os.listdir(train_path)

In [ ]:
len(os.listdir(test_path))

**Let's check how many images there are.**

In [ ]:
print("There are ", len(os.listdir(train_path+'/dog')), "dog images in the training dataset")     
print("There are ", len(os.listdir(train_path+'/cat')), "cat images in the training dataset")     

In [ ]:
os.listdir(train_path+'/cat')[0]

In [ ]:
path1=train_path+'/cat'+'/cat.0.jpg'
path2=train_path+'/dog'+'/dog.1.jpg'

In [ ]:
cat_img=imread(path1)
dog_img=imread(path2)

In [ ]:
plt.imshow(cat_img)

In [ ]:
plt.imshow(dog_img)

**Let's decide on the final dimension of these images.**

In [ ]:
cat_img.max()

In [ ]:
cat_img.min()

In [ ]:
cat_img.shape

In [ ]:
dog_img.shape

In [ ]:
x= []
y = []
for image in os.listdir(train_path+'/cat'):
    
    img = imread(train_path+'/cat/'+image)
    d1,d2,colors = img.shape
    x.append(d1)
    y.append(d2)

In [ ]:
sns.scatterplot(x=x,y=y)

In [ ]:
np.mean(x)

In [ ]:
np.mean(y)

In [ ]:
image_shape = (256,256,3)

In [ ]:
# help(ImageDataGenerator)

**Image Manipulation**

We can use the ImageDataGenerator to manipulate the images with rotation, resizing, and scaling so the model becomes more robust to different images that our data set doesn't have. ImageDataGenerator does the followings.

* Accepts a batch of images used for training.
* Applies a series of random transformations to each image in the batch.
* Replaces the original batch with randomly transformed batch.
* Training the CNN on this randomly transformed batch.

The goal of applying data augmentation is to have a more generalized model.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
image_gen = ImageDataGenerator(rotation_range=20, # rotate the image 20 degrees
                               width_shift_range=0.10, # Shift the pic width by a max of 5%
                               height_shift_range=0.10, # Shift the pic height by a max of 5%
                               rescale=1/255, # Rescale the image by normalzing it.
                               shear_range=0.1, # Shear means cutting away part of the image (max 10%)
                               zoom_range=0.1, # Zoom in by 10% max
                               horizontal_flip=True, # Allo horizontal flipping
                               fill_mode='nearest' # Fill in missing pixels with the nearest filled value
                              )

__flow_from_directory__ ile train ve test klasorlerinin altindaki resimleri okuduk. path bilgisini train' in icine kadar verdik, alt klasörleri kendisi okudu. Bu fonksiyon hangi resmi hangi klasörün altinda okuduysa bu bilgiyi aklinda tutar. Cikti olarak kac class ve kac image var bilgisini verdi :

__ImageDataGenerator__ resimlerimizi okudu fakat memory' sine atmadi cunku cok fazla resim var ve buna hafizasi yetmez. Yani sadece resimlerin isimlerini, class' larini ve nerede olduklarini biliyor. Her batch icin sırası geldiginde okuma islemini yapacak.

### Generating many manipulated images from a directory


flow_from_directory function works with images organized in sub-directories. Your directories should include only one class of images, so one folder per class of images.

main_directory/

.class_a/

......a_image_1.jpg

......a_image_2.jpg

.class_b/

......b_image_1.jpg

......b_image_2.jpg

In [ ]:
image_gen.flow_from_directory(val_path)

In [ ]:
image_gen.flow_from_directory(test_path)

# Creating the Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D

In [ ]:
#create your CNN model here
model = Sequential()

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=image_shape, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())


model.add(Dense(128, activation='relu'))

model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))


model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.summary()

# Early Stopping

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
early_stop = EarlyStopping(monitor='val_loss',patience=5)

In [ ]:
batch_size = 32

In [ ]:
train_image_gen = image_gen.flow_from_directory(train_path,
                                               target_size=image_shape[:2],
                                                color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='binary', shuffle=True)

In [ ]:
val_image_gen = image_gen.flow_from_directory(val_path,
                                               target_size=image_shape[:2],
                                               color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='binary',shuffle=True)

In [ ]:
train_image_gen.class_indices

In [ ]:
train_image_gen

# Model Training

In [ ]:
STEP_SIZE_TRAIN=train_image_gen.n//train_image_gen.batch_size
STEP_SIZE_VAL=val_image_gen.n//val_image_gen.batch_size

In [ ]:
STEP_SIZE_TRAIN, STEP_SIZE_VAL

In [ ]:
results = model.fit(train_image_gen,epochs=20,
                    validation_data=val_image_gen,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_steps=STEP_SIZE_VAL,
                    callbacks=[early_stop])

# Evaluating the Model

In [ ]:
summary = pd.DataFrame(model.history.history)
summary.head()

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(summary.loss, label="loss")
plt.plot(summary.val_loss, label="val_loss")
plt.legend(loc="upper right")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(summary.accuracy, label="accuracy")
plt.plot(summary.val_accuracy, label="val_accuracy")
plt.legend(loc="upper left")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.show()

In [ ]:
model.metrics_names

In [ ]:
val_image_gen = image_gen.flow_from_directory(val_path,
                                               target_size=image_shape[:2],
                                               color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='binary',shuffle=False, seed=42)

In [ ]:
pred_probabilities = model.predict(val_image_gen)

In [ ]:
pred_probabilities

In [ ]:
val_image_gen.classes[:50]

In [ ]:
predictions = pred_probabilities > 0.5

In [ ]:
predictions[:30]

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
print(classification_report(val_image_gen.classes,predictions))

In [ ]:
confusion_matrix(val_image_gen.classes,predictions)

In [ ]:
from tensorflow.keras.models import load_model
model.save('cat_dog_detector.h5')

# Predicting on an Image

In [ ]:
from tensorflow.keras.preprocessing import image

In [ ]:
model=load_model('cat_dog_detector.h5')

In [ ]:
from PIL import Image

In [ ]:
img=image.load_img('../input/iujpeg/iu.jpeg')
img

In [ ]:
resized_img = img.resize((256, 256))
resized_img

In [ ]:
resized_img1=np.array(resized_img)
resized_img1.shape

In [ ]:
resized_img1=np.expand_dims(resized_img1, axis=0)
resized_img1.shape

In [ ]:
model.predict(resized_img1).round()

In [ ]:
val_image_gen.class_indices